In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
import nltk
import chart_studio
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))
from bs4 import BeautifulSoup
import plotly.graph_objects as go
import chart_studio.plotly as py
import cufflinks
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
from chart_studio.plotly import iplot
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')


In [2]:
df = pd.read_csv('HADOOP_aug2_word_ori.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12304 entries, 0 to 12303
Data columns (total 41 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   text                    12304 non-null  object 
 1   labels                  12304 non-null  object 
 2   component               12304 non-null  object 
 3   auto-failover           12304 non-null  int64  
 4   azure                   12304 non-null  int64  
 5   benchmarks              12304 non-null  int64  
 6   bin                     12304 non-null  int64  
 7   build                   12304 non-null  int64  
 8   conf                    12304 non-null  int64  
 9   contrib/cloud           12304 non-null  int64  
 10  contrib/eclipse-plugin  12304 non-null  int64  
 11  contrib/hod             12304 non-null  int64  
 12  contrib/serialization   12304 non-null  int64  
 13  documentation           12304 non-null  int64  
 14  filecache               12304 non-null

In [4]:
print(df.component.value_counts())

fs               1632
build            1386
security         1200
test              948
documentation     856
                 ... 
azure,tools         2
nfs,test            2
build,yetus         2
fs,native           2
conf,security       2
Name: component, Length: 183, dtype: int64


In [5]:
'''# fs, build, security, test, documentation, ipc, io, conf, util, scripts, fs/s3, metrics, native, contrib/hod, yetus, tools, record, ha, net, kms, bin
df.loc[df['component'] == 'fs/swift,test', 'component'] = 'test'
df.loc[df['component'] == 'fs/s3,io', 'component'] = 'io'
df.loc[df['component'] == 'conf,ha', 'component'] = 'conf'
df.loc[df['component'] == 'build,scripts,tools,tools/distcp', 'component'] = 'build'
df.loc[df['component'] == 'benchmarks,conf,metrics', 'component'] = 'conf'
df.loc[df['component'] == 'ha,net', 'component'] = 'ha'
df.loc[df['component'] == 'fs,ipc,security', 'component'] = 'fs'
df.loc[df['component'] == 'documentation,site', 'component'] = 'site'
df.loc[df['component'] == 'conf,ha', 'component'] = 'ha'
df.loc[df['component'] == 'fs,ha', 'component'] = 'ha'
df.loc[df['component'] == 'native,test', 'component'] = 'native'
df.loc[df['component'] == 'conf,fs,fs/s3,ha,io,metrics,performance,record,security,util', 'component'] = 'ha'
#df.loc[df['component'] == 'Credit reporting', 'component'] = 'Credit reporting, credit repair services, or other personal consumer reports'
#df.loc[df['component'] == 'Credit card', 'component'] = 'Credit card or prepaid card'
#df.loc[df['component'] == 'Payday loan', 'component'] = 'Payday loan, title loan, or personal loan'
#df.loc[df['component'] == 'Virtual currency', 'component'] = 'Money transfer, virtual currency, or money service'
#df = df[df.component != 'Other financial service']'''

"# fs, build, security, test, documentation, ipc, io, conf, util, scripts, fs/s3, metrics, native, contrib/hod, yetus, tools, record, ha, net, kms, bin\ndf.loc[df['component'] == 'fs/swift,test', 'component'] = 'test'\ndf.loc[df['component'] == 'fs/s3,io', 'component'] = 'io'\ndf.loc[df['component'] == 'conf,ha', 'component'] = 'conf'\ndf.loc[df['component'] == 'build,scripts,tools,tools/distcp', 'component'] = 'build'\ndf.loc[df['component'] == 'benchmarks,conf,metrics', 'component'] = 'conf'\ndf.loc[df['component'] == 'ha,net', 'component'] = 'ha'\ndf.loc[df['component'] == 'fs,ipc,security', 'component'] = 'fs'\ndf.loc[df['component'] == 'documentation,site', 'component'] = 'site'\ndf.loc[df['component'] == 'conf,ha', 'component'] = 'ha'\ndf.loc[df['component'] == 'fs,ha', 'component'] = 'ha'\ndf.loc[df['component'] == 'native,test', 'component'] = 'native'\ndf.loc[df['component'] == 'conf,fs,fs/s3,ha,io,metrics,performance,record,security,util', 'component'] = 'ha'\n#df.loc[df['Pro

In [6]:
for x in range(len(df.component)):
    df.component[x] = df.component[x].split(',')[0]

In [7]:
print(df.component.value_counts())

fs                        1800
build                     1568
security                  1262
test                       970
documentation              954
ipc                        750
io                         710
conf                       616
scripts                    532
util                       530
fs/s3                      464
metrics                    340
native                     272
contrib/hod                228
yetus                      154
tools                      142
ha                         136
kms                         96
net                         86
record                      86
bin                         66
contrib/cloud               64
tools/distcp                56
nfs                         52
viewfs                      48
fs/swift                    48
auto-failover               46
benchmarks                  42
performance                 34
azure                       30
contrib/eclipse-plugin      26
filecache                   22
site    

In [8]:
df['component'].value_counts().sort_values(ascending=False).iplot(kind = 'bar', yTitle = 'Number of Complaints', title = 'Number complaints in each component')

In [9]:
print(df.loc[1])

text                      make Configuration an interface The Configurat...
labels                    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
component                                                              conf
auto-failover                                                             0
azure                                                                     0
benchmarks                                                                0
bin                                                                       0
build                                                                     0
conf                                                                      1
contrib/cloud                                                             0
contrib/eclipse-plugin                                                    0
contrib/hod                                                               0
contrib/serialization                                                     0
documentatio

In [10]:
def print_plot(index):
    example = df[df.index == index][['text', 'component']].values[0]
    if len(example) > 0:
        print(example[0])
        print('component:', example[1])
print_plot(2)

DF enhancement: performance and win XP support 1. DF is called twice for each heartbeat, which happens each 3 seconds. There is a simple fix for that in the attached patch.  2. cygwin is required to run df program in windows environment. There is a class org.apache.commons.io.FileSystemUtils, which can return disk free space for different OSs, but it does not have means to get disk capacity. In general in windows there is no efficient and uniform way to calculate disk capacity using a shell command. The choices are 'chkdsk' and 'defrag -a', but both of them are too slow to be called every 3 seconds. WinXP and 2003 server have a new tool called fsutil, which provides all necessary info. I implemented a call to fsutil in case df fails, and the OS is right. Other win versions should still run cygwin. I tested this fetaure for linux, winXP and cygwin. See attached patch.
component: fs


In [12]:
df = df.reset_index(drop=True)
df = df.astype(str)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))


def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
df['text'] = df['text'].apply(clean_text)
df['text'] = df['text'].str.replace('\d+', '')

D:\Users\PiusHwang\anaconda3\lib\site-packages\ipykernel_launcher.py:22: FutureWarning:

The default value of regex will change from True to False in a future version.



In [27]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['text'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 29698 unique tokens.


In [28]:
X = tokenizer.texts_to_sequences(df['text'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('X Shape of data tensor:', X.shape)

Y = pd.get_dummies(df['component']).values
print('Y Shape of label tensor:', Y.shape)

X Shape of data tensor: (12304, 250)
Y Shape of label tensor: (12304, 37)


In [29]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(11073, 250) (11073, 37)
(1231, 250) (1231, 37)


In [30]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(37, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])


In [31]:
print(X_train.shape)
print(Y_train.shape)

(11073, 250)
(11073, 37)


In [32]:
epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])


Epoch 1/5
156/156 [==============================] - 73s 470ms/step - loss: 2.9277 - accuracy: 0.1469 - val_loss: 2.8206 - val_accuracy: 0.2274
Epoch 2/5
156/156 [==============================] - 73s 466ms/step - loss: 2.3649 - accuracy: 0.3533 - val_loss: 1.9788 - val_accuracy: 0.4630
Epoch 3/5
156/156 [==============================] - 74s 471ms/step - loss: 1.4711 - accuracy: 0.6222 - val_loss: 1.5635 - val_accuracy: 0.5803
Epoch 4/5
156/156 [==============================] - 73s 470ms/step - loss: 0.9590 - accuracy: 0.7596 - val_loss: 1.2364 - val_accuracy: 0.6832
Epoch 5/5
156/156 [==============================] - 74s 474ms/step - loss: 0.6022 - accuracy: 0.8483 - val_loss: 1.1148 - val_accuracy: 0.7157


In [33]:
!pip install nbconvert

distutils: D:\Users\PiusHwang\anaconda3\Include\UNKNOWN
sysconfig: D:\Users\PiusHwang\anaconda3\Include
user = False
home = None
root = None
prefix = None
distutils: D:\Users\PiusHwang\anaconda3\Include\UNKNOWN
sysconfig: D:\Users\PiusHwang\anaconda3\Include
user = False
home = None
root = None
prefix = None
